# Scraping reviews from Reclame Aqui website

## Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
#import matplotlib.pyplot as plt
#%matplotlib inline


## Create driver

In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

## Create Url

In [3]:
def create_url(company, page):
    return f'https://www.reclameaqui.com.br/empresa/{company}/lista-reclamacoes/?pagina={page}'

In [4]:
company = 'upnid'
page = 2
url = create_url(company, page)
url

'https://www.reclameaqui.com.br/empresa/upnid/lista-reclamacoes/?pagina=2'

## Get complaints

In [31]:
def get_html(driver, url):
    begin = time.time()
    driver.get(url)
    end = time.time()
    if end - begin >= 1:
        driver.refresh()
        script = ("return document.getElements"
        "ByTagName('html')[0].innerHTML")
        html = driver.execute_script(script)
    return html

In [38]:
def get_html_2(driver, url):
    driver.get(url)
    script = ("return document.getElements"
    "ByTagName('html')[0].innerHTML")
    html = driver.execute_script(script)
    return html

In [10]:
def get_complaints_chunk(html):
    soup = BeautifulSoup(html, "html.parser")
    complaints_chunk = soup.find_all(class_="link-complain-id-complains")
    return complaints_chunk

In [66]:
def get_complaint_titles(html):
    title_list = []
    soup = BeautifulSoup(html, "html.parser")
    titles = soup.find_all('p', class_="text-title")
    for i in range(len(titles)):
        title_list.append(titles[i]['title'])
        
    return title_list

In [17]:
def get_complaint_title(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup.find_all('h1', class_="ng-binding")[0].contents[0]

In [24]:
def get_complaint_content(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup.find_all('p', class_="ng-binding")[9].contents[0]

In [25]:
def get_complaint_url(complaints_chunk,num_complaint):
    start_url = 'https://www.reclameaqui.com.br'
    return start_url + complaints_chunk[num_complaint].get('href')

In [11]:
html = get_html(driver,url)

In [65]:
get_complaint_titles(html)

[<p class="text-title ng-binding" title="Compra de empresa que não responde contato ">Compra de empresa que não responde contato </p>,
 <p class="text-title ng-binding" title="Reembolço do aparelho tv box">Reembolço do aparelho tv box</p>,
 <p class="text-title ng-binding" title="Pagamento sem retorno">Pagamento sem retorno</p>,
 <p class="text-title ng-binding" title="Pedido #3593 TV-Global Não Entregue** Sem localizador">Pedido #3593 TV-Global Não Entregue** Sem localizador</p>,
 <p class="text-title ng-binding" title="Produto não entregue">Produto não entregue</p>,
 <p class="text-title ng-binding" title="Ressarcir meu valor">Ressarcir meu valor</p>,
 <p class="text-title ng-binding" title="Compra">Compra</p>,
 <p class="text-title ng-binding" title="Produto não entregue">Produto não entregue</p>,
 <p class="text-title ng-binding" title="Compra de uma cadeirinha musical">Compra de uma cadeirinha musical</p>,
 <p class="text-title ng-binding" title="Compra">Compra</p>]

In [43]:
html

'<head><style type="text/css">[uib-typeahead-popup].dropdown-menu{display:block;}</style><style type="text/css">.uib-time input{width:50px;}</style><style type="text/css">[uib-tooltip-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-bottom > .tooltip-arrow,[uib-tooltip-popup].tooltip.right-top > .tooltip-arrow,[uib-tooltip-popup].tooltip.right-bottom > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.left-bottom > .tooltip-arrow,[uib-tooltip-htm

In [12]:
complaints_chunk = get_complaints_chunk(html)

In [13]:
len(complaints_chunk)

10

In [32]:
url_complaint = get_complaint_url(complaints_chunk,7)

In [33]:
html_complaint = get_html(driver,url_complaint)

In [34]:
get_complaint_title(html_complaint)

'Produto não entregue'

In [35]:
get_complaint_content(html_complaint)

'Comprei um processador de alimentos na promoção COMPRE 1 LEVE 2 em 24/02 no valor de R$ 54,80. Até o momento não recebi o produto e nem código de rastreamento.Aguardo uma solução.'

In [36]:
def get_titles_and_content(conplaints_chunk):
    titles = []
    contents = []
    for i in range(len(conplaints_chunk)):
        url_complaint = get_complaint_url(complaints_chunk,i)
        html_complaint = get_html(driver,url_complaint)
        title = get_complaint_title(html_complaint)
        content = get_complaint_content(html_complaint)
        titles.append(title)
        contents.append(content)
    return titles,contents